In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score 

In [53]:
data = pd.read_csv("train.csv",
                   names=[
                       "age", "workclass", "fnlwgt", "education",
                       "education-num", "marital-status", "occupation",
                       "relationship", "race", "sex", "capital-gain",
                       "capital-loss", "hours-per-week", "native-country",
                       "output"
                   ])
li = [
    "workclass", "education", "marital-status", "occupation", "relationship",
    "race", "sex", "native-country", "output"
]
for i in li:
    data[i] = data[i].astype('category')
    data[i] = data[i].cat.codes
y = data.iloc[:, 14]
x = data.iloc[:, 0:14]
y = y.to_numpy()
x_norm = (x - x.mean()) / (x.max() - x.min())
x1 = x_norm.to_numpy()
x = np.insert(x1, 0, 1, axis=1)
data = pd.read_csv("test.csv",
                   names=[
                       "age", "workclass", "fnlwgt", "education",
                       "education-num", "marital-status", "occupation",
                       "relationship", "race", "sex", "capital-gain",
                       "capital-loss", "hours-per-week", "native-country",
                       "output"
                   ])
for i in li:
    data[i] = data[i].astype('category')
    data[i] = data[i].cat.codes
y_test = data.iloc[:, 14]
x_test = data.iloc[:, 0:14]
y_test = y_test.to_numpy()
x_norm = (x_test - x_test.mean()) / (x_test.max() - x_test.min())
x1 = x_norm.to_numpy()
x_test = np.insert(x1, 0, 1, axis=1)

In [54]:
def acc_sig(z):
    return 1 / (1 + np.exp(0.5 - z))

In [55]:
def sigmoid(z):
    return 1 / (1 + np.exp(0 - z))

In [ ]:
def loss():
    

In [56]:
def accuracy(x, y, theta):
    ans = x @ theta.T
#     num = 0
#     for i in range(y.shape[0]):
#         if (ans[i]>=0.5):
#             ans[i][0]=1
#         else:
#             ans[i][0]=0
#         if (y[i]==ans[i][0]):
#             num+=1
#     return (num*100)/y.shape[0]
    ans = acc_sig(ans)
    ans = np.round(ans)
    y = np.reshape(y, (-1, 1))
    ans = ans-y
    num = np.sum(np.abs(ans))
    return ((y.shape[0]-num)*100)/y.shape[0]

In [71]:
def regression(x, y, alpha, x_test, y_test,type="normal",L=0):
    epoch = 10000
    theta = np.ones((1, 15))
    y = np.reshape(y, (-1, 1))
    if (type=="normal"):
        for k in range(epoch):
            z = x @ theta.T
            h = sigmoid(z)
            t = x.T @ (h - y)
            grad = (t) / y.shape[0]
            theta -= alpha * grad.T
    else:
        rms_arr = [0] * epoch
        rms_validation = [0] * epoch
        if (type=="l1"):
            for k in range(epoch):
                z = x @ theta.T
                h = sigmoid(z)
                t = x.T @ (h - y)
                grad = (t) / y.shape[0]
                grad += 2*L*np.sum(theta/np.abs(theta))
                theta -= alpha * grad.T
                rms_arr[k] = accuracy(x, y, theta)
                rms_validation[k] = accuracy(x_test, y_test, theta)
        elif (type=="l2"):
            for k in range(epoch):
                z = x @ theta.T
                h = sigmoid(z)
                t = x.T @ (h - y)
                grad = (t) / y.shape[0]
                grad+= L*np.sum(theta)
                theta -= alpha * grad.T
                rms_arr[k] = accuracy(x, y, theta)
                rms_validation[k] = accuracy(x_test, y_test, theta)
        print(rms_arr[-1],'final accuracy on train')
        print(rms_validation[-1],'final accuracy on test')
        plt.plot(rms_arr,'r')
#         plt.plot(100-rms_arr,'b')
        plt.title('Accuracy vs epoch for Training set'+str(type))
        plt.ylabel('Accuracy')
        plt.xlabel('epoch')
        plt.show()
        plt.plot(rms_validation,'r')
#         plt.plot(100-rms_validation,'b')
        plt.title('Accuracy vs epoch for Testing set'+str(type))
        plt.ylabel('Accuracy')
        plt.xlabel('epoch')
        plt.show()
    return theta

In [72]:
learning_rate = 0.05
theta = regression(x, y, learning_rate, x_test, y_test)
print(theta)

[[-1.50056875  2.57088091 -0.35730667  0.87174288  0.3265688   4.35030461
  -1.23461643  0.06485483 -0.67291787  0.45978707  0.86102584  2.63753076
   1.86697378  2.2495177   0.24265082]]


In [73]:
print(accuracy(x_test,y_test,theta))

79.26294820717132


In [ ]:
alphas = np.logspace(-10,0,100)
model = Lasso()
grid = GridSearchCV(estimator=model,param_grid=dict(alpha=alphas),cv=5)
grid.fit(x,y)
L = grid.best_estimator_.alpha
print(learning_rate ,L)
theta = regression(x,y,learning_rate,x_test,y_test,'l1',L)
print('Parameter vector Regression with Lasso l1 =>')
print(theta.T)

0.05 6.892612104349695e-07


In [ ]:
alphas = np.logspace(-10,0,100)
model = Ridge()
grid = GridSearchCV(estimator=model,param_grid=dict(alpha=alphas),cv=5)
grid.fit(x,y)
L = grid.best_estimator_.alpha
print(learning_rate ,L)
theta = regression(x,y,learning_rate,x_test,y_test,'l2',L)
print('Parameter vector Regression with Ridge l2 =>')
print(theta.T)